In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import os

In [2]:
output_path = "../../data-ceph/arguana/argmining22-sharedtask/models/"

#### Prepare the files for training:

In [3]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')

taska_training_df.insert(loc=0,column='row_num',value=np.arange(len(taska_training_df)))
taska_valid_df.insert(loc=0,column='row_num',value=np.arange(len(taska_valid_df)))

# taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '[CLS] {}:{} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
# taska_valid_df['input_txt'] = taska_valid_df.apply(lambda x: '[CLS] {}:{} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '{}:{}  </s></s> {} '.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_valid_df['input_txt'] = taska_valid_df.apply(lambda x: '{}:{} </s></s> {}'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)


taska_training_df['Premise'] = taska_training_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)

taska_validity_train_df = taska_training_df[taska_training_df.Validity != 0].copy()
taska_validity_valid_df = taska_valid_df[taska_valid_df.Validity != 0].copy()

taska_validity_train_df['label'] = taska_validity_train_df.Validity.apply(lambda x : 1 if x == 1 else 0)
taska_validity_valid_df['label'] = taska_validity_valid_df.Validity.apply(lambda x : 1 if x == 1 else 0)

taska_novelty_train_df = taska_training_df[taska_training_df.Novelty != 0].copy()
taska_novelty_valid_df = taska_valid_df[taska_valid_df.Novelty != 0].copy()

taska_novelty_train_df['label'] = taska_novelty_train_df.Novelty.apply(lambda x : 1 if x == 1 else 0)
taska_novelty_valid_df['label'] = taska_novelty_valid_df.Novelty.apply(lambda x : 1 if x == 1 else 0)

In [4]:
taska_validity_train_df[['row_num',  'label', 'input_txt']].to_csv('../data/multitask_data/validity_training_df.tsv', sep='\t', header=False, index=False)
taska_validity_valid_df[['row_num', 'label', 'input_txt' ]].to_csv('../data/multitask_data/validity_valid_df.tsv', sep='\t', header=False, index=False)

taska_novelty_train_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_training_df.tsv', sep='\t', header=False, index=False)
taska_novelty_valid_df[['row_num', 'label', 'input_txt']].to_csv('../data/multitask_data/novelty_valid_df.tsv', sep='\t', header=False, index=False)

### Train BERT model:

In [5]:
! python ../../multi-task-NLP/data_preparation.py \
  --task_file ../data/multitask_data/bert_based_task.yml \
  --data_dir ../data/multitask_data/ \
  --max_seq_len 512

task object created from task file...
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/curren

In [13]:
! python ../../multi-task-NLP/train.py \
      --data_dir ../data/multitask_data/bert-base-uncased_prepared_data/ \
      --task_file ../data/multitask_data/bert_based_task.yml \
      --learning_rate 5e-3 \
      --out_dir ../data/multitask_data/bert_model/ \
      --epochs 3 \
      --train_batch_size 8 \
      --eval_batch_size 16 \
      --grad_accumulation_steps 2 \
      --max_seq_len 512 \
      --log_per_updates 250 \
      --log_dir ../data/multitask_data/logs/ \
      --limit_save 15\
      --eval_while_train

INFO - logger created.
INFO - ARGS : {'data_dir': '../data/multitask_data/bert-base-uncased_prepared_data/', 'task_file': '../data/multitask_data/bert_based_task.yml', 'out_dir': '../data/multitask_data/bert_model/', 'epochs': 3, 'freeze_shared_model': False, 'train_batch_size': 8, 'eval_batch_size': 16, 'grad_accumulation_steps': 2, 'num_of_warmup_steps': 0, 'learning_rate': 0.005, 'epsilon': 1e-08, 'grad_clip_value': 1.0, 'log_file': 'multi_task_logs.log', 'log_dir': '../data/multitask_data/logs/', 'log_per_updates': 250, 'seed': 42, 'max_seq_len': 512, 'save_per_updates': 0, 'limit_save': 15, 'load_saved_model': None, 'eval_while_train': True, 'test_while_train': False, 'resume_train': False, 'finetune': False, 'debug_mode': False, 'silent': False}
INFO - Task params object created from task file...
INFO - task parameters:
 {'TaskA': {'config_name': 'bert-base-uncased', 'dropout_prob': 0.3, 'file_names': ['novelty_training_df.tsv', 'novelty_valid_df.tsv'], 'label_map_or_file': [1, 0

### Train NLI Model:

In [ ]:
! python ../../multi-task-NLP/data_preparation.py \
  --task_file ../data/multitask_data/nli_based_task.yml \
  --data_dir ../data/multitask_data/ \
  --max_seq_len 512

In [ ]:
! python ../../multi-task-NLP/train.py \
      --data_dir ../data/multitask_data/roberta-large-mnli_prepared_data/ \
      --task_file ../data/multitask_data/nli_based_task.yml \
      --learning_rate 2e-5 \
      --out_dir ../data/multitask_data/nli_model/ \
      --epochs 3 \
      --train_batch_size 8 \
      --eval_batch_size 8 \
      --grad_accumulation_steps 2 \
      --max_seq_len 512 \
      --log_per_updates 100 \
      --log_dir /var/tmp/argsvalidnovel/multitask \
      --limit_save 15\
      --eval_while_train

### Perform prediction:

In [ ]:
%load_ext autoreload

In [ ]:
import sys
import glob
import json
sys.path.append('../../multi-task-NLP/')
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
%autoreload
from infer_pipeline import inferPipeline

In [ ]:
bert_model = inferPipeline(modelPath = '../data/multitask_data/bert_model/multi_task_model_10_995.pt', maxSeqLen = 512)
nli_model  = inferPipeline(modelPath = '../data/multitask_data/nli_model/multi_task_model_9_905.pt', maxSeqLen = 512)

In [ ]:
bert_preds = bert_model.infer(taska_valid_df['input_txt'].tolist(), ['TaskA', 'TaskB']) 
nli_preds  = nli_model.infer(taska_valid_df['input_txt'].tolist(), ['TaskA', 'TaskB']) 

In [ ]:
taska_valid_df['bert_pred_validity'] = [x['TaskB'][0] for x in bert_preds]
taska_valid_df['bert_pred_novelty']  = [x['TaskA'][0] for x in bert_preds]
taska_valid_df['nli_pred_validity']  = [x['TaskB'][0] for x in nli_preds]
taska_valid_df['nli_pred_novelty']   = [x['TaskA'][0] for x in nli_preds]

In [ ]:
print('BERT Validity: ', f1_score(taska_valid_df.Validity.tolist(), taska_valid_df.bert_pred_validity.tolist(), average='macro'))
print('BERT Novelty: ', f1_score(taska_valid_df.Novelty.tolist(), taska_valid_df.bert_pred_novelty.tolist(), average='macro'))

print('NLI Validity: ', f1_score(taska_valid_df.Validity.tolist(), taska_valid_df.nli_pred_validity.tolist(), average='macro'))
print('NLI Novelty: ', f1_score(taska_valid_df.Novelty.tolist(), taska_valid_df.nli_pred_novelty.tolist(), average='macro'))